# SAE 601 - Développement et test d'un outil décisionnel

## Présentation du projet

Ce document à été crée dans l'objectif de réaliser un outil décisionel d'**Analyse de la métagame** (meilleurs winrates, meilleurs combinaison de decks, meilleurs anti-métas, etc...) du jeu "**Pokémon TCG : Pocket**". 
<br>Cette analyse à été effectué via des **données scrappés** sur le site **https://limitlesstcg.com** à propos des tournois organisés sur le jeu.

Il contient donc :
- L'import des données via une base de donnée PostgreSQL
- Une mise en forme et transformations des données d'entrée
- Des analyses statistiques de ces mêmes données
- Des analyses graphiques permettant une visualisation claire de la méta

## Import des packages

In [5]:
### Installation ddes packages (optionel si ceux-ci sont déjà installés)
#!pip install psycopg
#!pip install numpy
#!pip install matplotlib
#!pip install pandas
#!pip install tqdm
#!pip install time
#!pip install seaborn
#!pip install logging

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 9.6 MB/s eta 0:00:00


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize 
from matplotlib.cm import ScalarMappable
import matplotlib.gridspec as gridspec
import pandas as pd
from tqdm import tqdm
import time
import os
import math   
import re
import seaborn as sns
import warnings
import logging
import psycopg
from psycopg.rows import dict_row
from typing import List, Dict, Any
from datetime import datetime

## Import des données via une **BDD PostgreSQL**

### Classe d'import des données depuis PostgreSQL

In [ ]:
class PostgresDataImporter:
    """
    Classe pour importer des données depuis une base de données PostgreSQL
    """
    
    def __init__(self, host: str, database: str, user: str, password: str, port: int = 5432):
        """
        Initialise la connexion à la base de données
        
        Args:
            host: Adresse du serveur PostgreSQL
            database: Nom de la base de données
            user: Nom d'utilisateur
            password: Mot de passe
            port: Port de connexion (défaut: 5432)
        """
        self.connection_params = {
            'host': host,
            'database': database,
            'user': user,
            'password': password,
            'port': port
        }
        self.connection = None
    
    def connect(self) -> bool:
        """
        Établit la connexion à la base de données
        
        Returns:
            bool: True si connexion réussie, False sinon
        """
        try:
            self.connection = psycopg.connect(**self.connection_params)
            print(f"Connexion établie à la base de données '{self.connection_params['database']}'")
            return True
        except psycopg.Error as e:
            print(f"Erreur de connexion: {e}")
            return False
    
    def disconnect(self):
        """Ferme la connexion à la base de données"""
        if self.connection:
            self.connection.close()
            print("Connexion fermée")
    
    def list_tables(self) -> List[str]:
        """
        Liste toutes les tables de la base de données
        
        Returns:
            List[str]: Liste des noms de tables
        """
        query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
        """
        
        try:
            with self.connection.cursor() as cursor:
                cursor.execute(query)
                tables = [row[0] for row in cursor.fetchall()]
                return tables
        except psycopg.Error as e:
            print(f"Erreur lors de la récupération des tables: {e}")
            return []
    
    def get_table_info(self, table_name: str) -> List[Dict[str, Any]]:
        """
        Récupère les informations sur les colonnes d'une table
        
        Args:
            table_name: Nom de la table
            
        Returns:
            List[Dict]: Informations sur les colonnes
        """
        query = """
        SELECT 
            column_name,
            data_type,
            is_nullable,
            column_default
        FROM information_schema.columns 
        WHERE table_name = %s
        ORDER BY ordinal_position;
        """
        
        try:
            with self.connection.cursor(row_factory=dict_row) as cursor:
                cursor.execute(query, (table_name,))
                return cursor.fetchall()
        except psycopg.Error as e:
            print(f"Erreur lors de la récupération des infos de la table: {e}")
            return []
    
    def import_data(self, query: str, params: tuple = None) -> List[Dict[str, Any]]:
        """
        Importe des données avec une requête SQL personnalisée
        
        Args:
            query: Requête SQL à exécuter
            params: Paramètres pour la requête (optionnel)
            
        Returns:
            List[Dict]: Données récupérées
        """
        try:
            with self.connection.cursor(row_factory=dict_row) as cursor:
                cursor.execute(query, params)
                data = cursor.fetchall()
                print(f"{len(data)} enregistrements importés")
                return data
        except psycopg.Error as e:
            print(f"Erreur lors de l'importation: {e}")
            return []
    
    def import_table(self, table_name: str, limit: int = None) -> List[Dict[str, Any]]:
        """
        Importe toutes les données d'une table
        
        Args:
            table_name: Nom de la table
            limit: Limite du nombre de lignes (optionnel)
            
        Returns:
            List[Dict]: Données de la table
        """
        query = f"SELECT * FROM {table_name}"
        if limit:
            query += f" LIMIT {limit}"
        
        return self.import_data(query)
    
    def to_dataframe(self, data: List[Dict[str, Any]]) -> pd.DataFrame:
        """
        Convertit les données en DataFrame pandas
        
        Args:
            data: Données à convertir
            
        Returns:
            pd.DataFrame: DataFrame pandas
        """
        return pd.DataFrame(data)
    
    def export_to_csv(self, data: List[Dict[str, Any]], filename: str):
        """
        Exporte les données vers un fichier CSV
        
        Args:
            data: Données à exporter
            filename: Nom du fichier CSV
        """
        df = self.to_dataframe(data)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Données exportées vers {filename}")


### Main

In [ ]:
def main():
    """Fonction principale - exemple d'utilisation"""
    
    # Configuration de la connexion
    # Vous pouvez aussi utiliser des variables d'environnement pour plus de sécurité
    DB_CONFIG = {
        'host': os.getenv('DB_HOST', 'localhost'),
        'database': os.getenv('DB_NAME', 'ma_base'),
        'user': os.getenv('DB_USER', 'mon_utilisateur'),
        'password': os.getenv('DB_PASSWORD', 'mon_mot_de_passe'),
        'port': int(os.getenv('DB_PORT', 5432))
    }
    
    # Création de l'importateur
    importer = PostgresDataImporter(**DB_CONFIG)
    
    try:
        # Connexion à la base
        if not importer.connect():
            return
        
        # Liste des tables disponibles
        print("\nTables disponibles:")
        tables = importer.list_tables()
        for i, table in enumerate(tables, 1):
            print(f"  {i}. {table}")
        
        if not tables:
            print("Aucune table trouvée")
            return
        
        # Exemple: Informations sur la première table
        first_table = tables[0]
        print(f"\nStructure de la table '{first_table}':")
        columns = importer.get_table_info(first_table)
        for col in columns:
            nullable = "NULL" if col['is_nullable'] == 'YES' else "NOT NULL"
            default = f" DEFAULT {col['column_default']}" if col['column_default'] else ""
            print(f"  - {col['column_name']}: {col['data_type']} {nullable}{default}")
        
        # Importation des données
        print(f"\nImportation des données de '{first_table}':")
        data = importer.import_table(first_table, limit=10)  # Limite à 10 lignes pour l'exemple
        
        if data:
            # Affichage des premières lignes
            print("\nAperçu des données:")
            df = importer.to_dataframe(data)
            print(df.head())
            
            # Export vers CSV
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filename = f"{first_table}_{timestamp}.csv"
            importer.export_to_csv(data, csv_filename)
        
        # Exemple de requête personnalisée
        print(f"\nExemple de requête personnalisée:")
        custom_query = f"SELECT COUNT(*) as total_records FROM {first_table}"
        result = importer.import_data(custom_query)
        if result:
            print(f"Nombre total d'enregistrements dans {first_table}: {result[0]['total_records']}")
    
    except Exception as e:
        print(f"Erreur générale: {e}")
    
    finally:
        # Fermeture de la connexion
        importer.disconnect()


if __name__ == "__main__":
    # Pour utiliser ce script, vous devez d'abord installer les dépendances:
    # pip install psycopg pandas
    
    # Puis configurer vos paramètres de connexion dans DB_CONFIG
    # ou définir les variables d'environnement suivantes:
    # DB_HOST, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT
    
    main()

## Data préparation

## Data visualisation